In [1]:
import pandas.io.sql as psql
import re
import psycopg2
from gensim.models.word2vec import Word2Vec
from sklearn.externals import joblib

In [2]:
import json
with open('/Users/krishna/MOOC/smart-city/login.json') as data_file:
    db = json.load(data_file)

conn = psycopg2.connect(database=db['dbname'], user=db['user'], password=db['password'], host=db['host'], port=db['port'])

In [3]:
dataframe = psql.read_sql("select text from twitter.tweets limit 1500000",conn)

In [4]:
dataframe.head()

,text
0,Peace activist Jesuit priest Daniel Berrigan d...
1,RT @srgdxddz: congratulations for your marriag...
2,RT @starfoutchip: C quel genre de pute ca enco...
3,Bavure policière à Lyon : R2D2 Cazeneuve gazé ...
4,Le défilé du #1ermai s'engage dans une ambianc...


In [13]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # Remove some punctuations
    tweet = re.sub(r'[-:.,]', '', tweet)
    tweet = re.sub('[^A-Za-z0-9]+', ' ', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    return tweet.split(" ")
x_train = (dataframe['text']).apply(lambda x : processTweet(x))
x_train.head()

0    [peace, activist, jesuit, priest, daniel, berr...
1    [rt, congratulations, for, your, marriage, and...
2    [rt, c, quel, genre, de, pute, ca, encore, ell...
3    [bavure, polici, re, lyon, r2d2, cazeneuve, ga...
4    [le, d, fil, du, 1ermai, s, engage, dans, une,...
Name: text, dtype: object

In [14]:
min_count = 1
size = 50
window = 5
model = Word2Vec(x_train, min_count=min_count, size=size, window=window)

In [18]:
model.most_similar(positive=['restaurants'])

[('terrasses', 0.7827770709991455),
 ('salles', 0.766467809677124),
 ('visiteurs', 0.7352768182754517),
 ('chefs', 0.7304201722145081),
 ('ats', 0.7221043109893799),
 ('dits', 0.7198870778083801),
 ('histoires', 0.71758633852005),
 ('passages', 0.7165863513946533),
 ('tablissements', 0.7121161222457886),
 ('mondiaux', 0.7073555588722229)]

In [17]:
model.similarity("paris","lyon")

0.58235292260779425

In [9]:
model['lyon']

array([ 2.49685001,  1.99363482, -1.25289023, -1.74603617,  0.75938785,
       -0.12584274, -2.54330206, -0.3748886 , -2.15562916, -0.88260007,
       -0.62843812,  3.60703015,  0.47680762,  2.70707059, -2.51631808,
       -1.45990252,  0.55457407,  1.29574287, -0.65043068, -2.20138836,
       -1.88914156,  3.602705  ,  1.00108051,  1.69884992,  1.8701781 ,
        2.26305795, -0.91946983, -0.68555307,  5.41026163, -0.90500998,
        4.968009  , -1.2310679 , -2.16619039, -4.69600391,  0.16414624,
       -0.41872534, -1.93912029, -0.62758362,  4.62916994,  1.68850708,
       -0.21018761, -1.13256812,  2.65498734,  1.34077191, -1.38609684,
        3.86183405,  0.52569824, -3.20377946, -0.14594096,  2.10592031], dtype=float32)

In [10]:
model.most_similar(positive=['lyon'], negative=['film'])

[('lille', 0.5551673769950867),
 ('hsterdyniak)', 0.5543833374977112),
 ('angers', 0.5434356927871704),
 ('dadoune', 0.5413603186607361),
 ('guingamp', 0.5376990437507629),
 ('+2\xc2\xae', 0.5227891802787781),
 ("j'comptait", 0.5224996209144592),
 ('sahibim', 0.5223608016967773),
 ('lyon)', 0.5164003968238831),
 ('lyon\xc2\xbb', 0.5089914798736572)]

In [20]:
model.save('tweetW2V')

In [21]:
model2 = Word2Vec.load('tweetW2V')

In [25]:
model2.most_similar('france')[1:3]

[('waarisjoost', 0.5983824133872986), ('pytagoque', 0.5957268476486206)]